In [1]:
import os
from collections import defaultdict

import numpy as np
import torch
import torchvision.transforms as T
from torchvision.datasets import ImageFolder
from torchvision.models import resnet50
from tqdm import tqdm

torch.manual_seed(0)
device = "cuda" if torch.cuda.is_available() else "cpu"
WEIGHT_PATH = "weights"
RECORD_PATH = "records"
os.makedirs(WEIGHT_PATH, exist_ok=True)
os.makedirs(RECORD_PATH, exist_ok=True)

In [2]:
transform_train = T.Compose(
    [
        T.RandomResizedCrop((224, 224)),
        T.RandAugment(),
        T.ToTensor(),
    ]
)

transform_eval = T.Compose(
    [
        T.Resize((224, 224)),
        T.ToTensor(),
    ]
)

trainset = ImageFolder("birds-400/train", transform=transform_train)
validset = ImageFolder("birds-400/valid", transform=transform_eval)
testset = ImageFolder("birds-400/test", transform=transform_eval)

model = resnet50()
model.fc = torch.nn.Sequential(
    torch.nn.Linear(2048, 512), torch.nn.ReLU(), torch.nn.Linear(512, 128)
)

model.load_state_dict(torch.load(os.path.join(WEIGHT_PATH, "r50_simclr_pretrain.pt")))

for p in model.parameters():
    p.requires_grad = False

model.fc = torch.nn.Linear(2048, len(trainset.class_to_idx))

model = model.to(device)

trainloader = torch.utils.data.DataLoader(trainset, 32, True, pin_memory=True)
validloader = torch.utils.data.DataLoader(validset, 32, pin_memory=True)
testloader = torch.utils.data.DataLoader(testset, 32, pin_memory=True)


optimizer = torch.optim.SGD(model.parameters(), 5e-3, momentum=0.9, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, len(trainloader) * 50)
criterion = torch.nn.CrossEntropyLoss()

In [3]:
history = []
for i in range(1, 51):
    print(f"Epoch {i}/50")
    metric = defaultdict(list)
    pbar = tqdm(total=len(trainloader))

    model.train()
    for i, (inputs, targets) in enumerate(trainloader, 1):
        inputs = inputs.to(device)
        targets = targets.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        metric["train_loss"].append(loss.item())
        metric["train_accuracy"].append(
            (outputs.argmax(-1) == targets).float().mean().item()
        )
        pbar.set_postfix({k: np.mean(v) for k, v in metric.items()})
        pbar.update()

    model.eval()
    for i, (inputs, targets) in enumerate(validloader, 1):
        inputs = inputs.to(device)
        targets = targets.to(device)

        with torch.inference_mode():
            outputs = model(inputs)
            loss = criterion(outputs, targets)

        metric["valid_loss"].append(loss.item())
        metric["valid_accuracy"].append(
            (outputs.argmax(-1) == targets).float().mean().item()
        )
        pbar.set_postfix({k: np.mean(v) for k, v in metric.items()})

    history.append({k: np.mean(v) for k, v in metric.items()})
    pbar.close()
torch.save(model.state_dict(), os.path.join(WEIGHT_PATH, "r50_simclr_finetune.pt"))
torch.save(history, os.path.join(RECORD_PATH, "r50_simclr_finetune.pt"))

Epoch 1/50


100%|██████████| 1825/1825 [03:34<00:00,  8.49it/s, train_loss=4.91, train_accuracy=0.112, valid_loss=4.12, valid_accuracy=0.21]  


Epoch 2/50


100%|██████████| 1825/1825 [03:35<00:00,  8.49it/s, train_loss=4.4, train_accuracy=0.169, valid_loss=3.9, valid_accuracy=0.222]  


Epoch 3/50


...

100%|██████████| 1825/1825 [03:34<00:00,  8.50it/s, train_loss=2.46, train_accuracy=0.456, valid_loss=2, valid_accuracy=0.541]   


Epoch 49/50


100%|██████████| 1825/1825 [03:35<00:00,  8.48it/s, train_loss=2.45, train_accuracy=0.46, valid_loss=2, valid_accuracy=0.545]   


Epoch 50/50


100%|██████████| 1825/1825 [03:35<00:00,  8.47it/s, train_loss=2.46, train_accuracy=0.46, valid_loss=2, valid_accuracy=0.542]   
